In [ ]:
# Define the connection properties
jdbc_url = "jdbc:sqlserver://lab3-sql-server.database.windows.net:1433;database=lab3-sql-db"
connection_properties = {
    "user": "sarankan",
    "password": "s123456789!",
    "driver": "com.microsoft.sqlserver.jdbc.SQLServerDriver"
}
 
# Define the table name
table_name = "STOCK"
 
# Read the table into a DataFrame
df = spark.read.jdbc(url=jdbc_url, table=table_name, properties=connection_properties)
 
# Show the DataFrame
df.show()

+----------+-----------------+-----------------+-----------------+-----------------+---------+-----------------+------------+----------+
|      Date|             High|              Low|             Open|            Close|   Volume|        Adj Close|company_name|stock_name|
+----------+-----------------+-----------------+-----------------+-----------------+---------+-----------------+------------+----------+
|2017-01-03| 758.760009765625|747.7000122070312|757.9199829101562|753.6699829101562|3521100.0|753.6699829101562|      AMAZON|      AMZN|
|2017-01-04|759.6799926757812|754.2000122070312|758.3900146484375|757.1799926757812|2510500.0|757.1799926757812|      AMAZON|      AMZN|
|2017-01-05|782.4000244140625| 760.260009765625|761.5499877929688|780.4500122070312|5830100.0|780.4500122070312|      AMAZON|      AMZN|
|2017-01-06|799.4400024414062|  778.47998046875|782.3599853515625| 795.989990234375|5986200.0| 795.989990234375|      AMAZON|      AMZN|
|2017-01-09|  801.77001953125|  791.77001

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.window import Window

def calculate_average_daily_return(stock_data, stock_name, start_date, end_date):
    # Filter stock data for the specified stock and date range
    filtered_data = stock_data.filter((F.col('stock_name') == stock_name) & (F.col('Date') >= start_date) & (F.col('Date') <= end_date))
 
    # Calculate daily return rate
    window_spec = Window().orderBy('Date')
    filtered_data = filtered_data.withColumn('Daily Return', (F.col('Adj Close') - F.lag('Adj Close').over(window_spec)) / F.lag('Adj Close').over(window_spec))
 
    # Drop the first row as it will have NaN for the daily return
    filtered_data = filtered_data.dropna()
 
    # Calculate average daily return
    average_daily_return = filtered_data.groupBy().agg(F.avg('Daily Return')).collect()[0][0]
 
    return average_daily_return
 
# Example usage:
# Specify the stock name, start date, and end date
stock_name = 'AMZN'
start_date = '2017-01-03'
end_date = '2017-01-05'
 
# Call the function
average_daily_return = calculate_average_daily_return(df, stock_name, start_date, end_date)
 
# Display the result
print(f"Average Daily Return for {stock_name} between {start_date} and {end_date}: {average_daily_return}")

Average Daily Return for AMZN between 2017-01-03 and 2017-01-05: 0.0032172458640818582


In [ ]:
def calculate_moving_average(stock_data, stock_name, start_date, end_date, window_size):
    # Filter stock data for the specified stock and date range
    filtered_data = stock_data.filter((F.col('stock_name') == stock_name) & (F.col('Date') >= start_date) & (F.col('Date') <= end_date))

    # Calculate moving average over the opening price column
    window_spec = Window().orderBy('Date').rowsBetween(-window_size, 0)
    filtered_data = filtered_data.withColumn('Moving Average', F.avg('Open').over(window_spec))

    return filtered_data.select('Date', 'Open', 'Moving Average')

# Example usage:
# Specify the stock name, start date, and end date
stock_name = 'AMZN'
start_date = '2017-01-03'
end_date = '2017-01-05'
window_size = 5

# Call the function
moving_average_data = calculate_moving_average(df, stock_name, start_date, end_date, window_size)

# Display the result
moving_average_data.show()

+----------+-----------------+-----------------+
|      Date|             Open|   Moving Average|
+----------+-----------------+-----------------+
|2017-01-03|757.9199829101562|757.9199829101562|
|2017-01-03|757.9199829101562|757.9199829101562|
|2017-01-03|757.9199829101562|757.9199829101562|
|2017-01-03|757.9199829101562|757.9199829101562|
|2017-01-04|758.3900146484375|758.0139892578125|
|2017-01-04|758.3900146484375|  758.07666015625|
|2017-01-04|758.3900146484375|758.1549987792969|
|2017-01-04|758.3900146484375|758.2333374023438|
|2017-01-05|761.5499877929688|758.8383382161459|
|2017-01-05|761.5499877929688|759.4433390299479|
|2017-01-05|761.5499877929688|759.9700012207031|
|2017-01-05|761.5499877929688|760.4966634114584|
+----------+-----------------+-----------------+



In [ ]:
#%pip install --upgrade azure-storage-blob

In [ ]:
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient, PublicAccess

account_name = "lab3stoaccount"
account_key = "jQ+o/g/0QzQbvKe6ScSdArB+zSWFqZMeB0dxL+ivOY3y2xt64OINcSGegNXvWqaXA2Qx2UTGXoEO+AStXxeH4w=="
account_url = f"https://{account_name}.blob.core.windows.net"
container_name = "output-databricks"

In [ ]:
blob_service = BlobServiceClient(credential=account_key , account_url = account_url)

container_client = blob_service.get_container_client(container_name)

In [ ]:
result1_str = f"Average Daily Return for {stock_name} between {start_date} and {end_date}: {average_daily_return}"

result1_outputfile = 'result1.txt'

In [ ]:
result2_str = moving_average_data.toPandas().to_string()

result2_outputfile = 'result2.txt'

In [ ]:
blob_client = container_client.get_blob_client(result1_outputfile)
blob_client.upload_blob(result1_str, overwrite=True)

blob_client = container_client.get_blob_client(result2_outputfile)
blob_client.upload_blob(result2_str, overwrite=True)

Out[46]: {'etag': '"0x8DBEE0A90FA0500"',
 'last_modified': datetime.datetime(2023, 11, 25, 23, 2, 15, tzinfo=datetime.timezone.utc),
 'content_md5': bytearray(b'\x8d|\x80\xdc\xea\xd1\xe71q7R\xdbR\x8f\x1f\x1f'),
 'client_request_id': 'acc3632e-8be6-11ee-9688-00163ec8f308',
 'request_id': '03a2d101-101e-002b-6bf3-1f8d40000000',
 'version': '2023-11-03',
 'version_id': None,
 'date': datetime.datetime(2023, 11, 25, 23, 2, 15, tzinfo=datetime.timezone.utc),
 'request_server_encrypted': True,
 'encryption_key_sha256': None,
 'encryption_scope': None}